In [ ]:
import os
import sys
sys.path.insert(1, '/mnt/d/PowerTAC/Python/python_utils/helper')  # For Jupyter notebook
# sys.path.insert(1, 'D:\PowerTAC\Python\python_utils\helper')  # For VS code

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go

from matplotlib import pyplot as plt
from plotly.subplots import make_subplots
from read_mongo_collection import HelperToReadMongo

# Adding background to the graph (few good backgrounds below)
plt.style.use("fivethirtyeight")
# plt.style.use("ggplot")
# plt.style.use("seaborn-*")

# See complete data
pd.set_option('max_columns', None)
pd.set_option("max_rows", None)
np.set_printoptions(threshold=np.inf)

In [ ]:
helper_mongo = HelperToReadMongo()

In [ ]:
rename_col = {"MCP": "Price", "LimitPrice": "Price" , "Net_Cleared_Quantity": "Quantity", \
              "Broker's_Cleared_Quantity": "Quantity", "Broker's_Bidded_Quantity": "Quantity"}

In [ ]:
color_book = {'VV21': 'darkorchid', 'VidyutVanika': 'slateblue', 'VV20': 'slateblue', 'VidyutVanika18': 'yellowgreen', 'VV18': 'yellowgreen', \
              'ZI': 'orchid', 'ZIP': 'slategray', 'SPOT': 'orangered', 'TUC_TAC': 'gold', 'MCP': 'red'}

# Data Processing

In [ ]:
# Specify names of all the databases here
list_of_databases = ['VV2110', 'VV1810', 'SPOT10', 'ZI10', 'ZIP10']

# Specify names of all the brokers corresponding to databases here
list_of_brokers = ['VV21', 'VV18', 'SPOT', 'ZI', 'ZIP']

# Specify list of the collections to be analysed here
# list_of_collections = helper_mongo.connect_to_mongo('PowerTAC2020_Wholesale_Test_VV21').list_collection_names()
list_of_collections = ['MarketTransaction_Info', 'Submitted_Bid_Information', 'ClearedTrade_Info']

# Specify the storage path along with directory
storage_path = '/mnt/d/PowerTAC/PowerTAC2021/experiments/Wholesale_Executables/0.10_net_demand'
# os.makedirs(storage_path, True)

# Keep all the collections at one place
dict_of_dataframes = dict()
dict_of_aggregated_lp = dict()
dict_of_aggregated_lq = dict()
# dict_of_aggregated_lp_sd = dict()
# dict_of_aggregated_lq_sd = dict()

In [ ]:
# View the Collections in all the databases

def view_collections(list_of_databases):
    
    for database in list_of_databases:
        
        db = helper_mongo.connect_to_mongo(database, server_ip='10.2.16.159', ssh_username='powertac', \
                                           ssh_password='sanjay9397', remote=True)
        print(database + "\n" + "-"*35)
        print(db.list_collection_names())
        print("\n")

# Function Calling
view_collections(list_of_databases)

In [ ]:
for dataset, broker in zip(list_of_databases, list_of_brokers):
    
    inner_dict = dict()
    inner_dict_lp = dict()
    inner_dict_lq = dict()
    # inner_dict_lp_sd = dict()
    # inner_dict_lq_sd = dict()
        
    for collection in list_of_collections:
        
        db = helper_mongo.query_to_mongo(dataset, collection, server_ip='10.2.16.159', ssh_username='powertac', \
                                         ssh_password='sanjay9397', remote=True)
        db.rename(columns = rename_col, inplace = True)
        
        if (collection != 'ClearedTrade_Info'):
            # consider only bids
            db = db[db['Price'] < 0.0]
            db = db[abs(db['Quantity']) > 0.001]
            
        db['Proximity'] = db['Execution_Timeslot'] - db['Bidding_Timeslot']
        groupby_db = db.groupby('Proximity')
        
        lp = abs(groupby_db['Price'].mean())
        lq = abs(groupby_db['Quantity'].sum())
        # lp_sd = groupby_db['Price'].std()
        # lq_sd = groupby_db['Quantity'].std()
                    
        inner_dict.update({collection: db})
        inner_dict_lp.update({collection: lp})
        inner_dict_lq.update({collection: lq})
        # inner_dict_lp_sd.update({collection: lp_sd})
        # inner_dict_lq_sd.update({collection: lq_sd})
                
    dict_of_dataframes.update({broker: inner_dict})
    dict_of_aggregated_lp.update({broker: inner_dict_lp})
    dict_of_aggregated_lq.update({broker: inner_dict_lq})
    # dict_of_aggregated_lp_sd.update({broker: inner_dict_lp_sd})
    # dict_of_aggregated_lq_sd.update({broker: inner_dict_lq_sd})


# Analysis and Plotting

In [ ]:
num_rows = 1
num_cols = 3

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(18,8))
fig.tight_layout(pad=5)
       
axes[0].set_title("Clearing-price Comparison")
axes[1].set_title("Limit-price Comparison")
axes[2].set_title("MCP Comparison")

for col in range(num_cols):
    axes[col].set_ylabel("Price")
    axes[col].set_xlabel('Proximity')
        
for broker in list_of_brokers:
    for col, i in zip(list_of_collections, range(num_cols)):
        data = dict_of_aggregated_lp.get(broker).get(col)
        # data_sd = dict_of_aggregated_lp_sd.get(broker).get(col)
        axes[i].plot(data, label=broker)
        # axes[i].fill_between(np.arange(1,data.shape[0]+1), data-data_sd, data+data_sd)

for col in range(num_cols):
    axes[col].legend()

plt.setp(axes, xlim=(0, 25))
plt.savefig(storage_path + "/overall1.jpg")
plt.close()

In [ ]:
num_rows = 1
num_cols = 3

fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols, figsize=(18,8))
fig.tight_layout(pad=5)
       
axes[0].set_title("Clearing-quantity Comparison")
axes[1].set_title("Limitquantity Comparison")

for col in range(num_cols-1):
    axes[col].set_ylabel("Quantity")
    axes[col].set_xlabel('Proximity')
        
updated_list_of_collections = list_of_collections.copy()
updated_list_of_collections.remove('ClearedTrade_Info')
        
for broker in list_of_brokers:
    for col, i in zip(updated_list_of_collections, range(num_cols-1)):            
            
        data = dict_of_aggregated_lq.get(broker).get(col)            
        axes[i].plot(data, label=broker)
        
axes[num_cols-1].set_title("MCP")
axes[num_cols-1].plot(dict_of_aggregated_lp.get('VV21').get('ClearedTrade_Info'), label="MCP")
axes[num_cols-1].set_xlabel("Proximity")

for col in range(num_cols):
    axes[col].legend()
    
plt.setp(axes, xlim=(0, 25))
plt.savefig(storage_path + "/overall2.jpg")
plt.close()

In [ ]:
# num_rows = 3
# num_cols = 2

# fig = make_subplots(rows=num_rows, 
#                     cols=num_cols, 
#                     subplot_titles=('Clearing-quantity Comparison', 'Clearing-price Comparison',
#                                     'Limit-quantity Comparison', 'Limit-price Comparison', 'MCP'))

# updated_list_of_collections = list_of_collections.copy()
# updated_list_of_collections.remove('ClearedTrade_Info')
        
# for broker in list_of_brokers:
#     for col, i in zip(updated_list_of_collections, range(1,num_rows)):            
            
#         data = dict_of_aggregated_lq.get(broker).get(col)     
#         fig.append_trace(go.Scatter(
#             x = np.arange(1,25), 
#             y = data, 
#             name = broker, 
#             line=dict(color=color_book.get(broker), width=3),
#             legendgroup = i
#         ), row = i, col = 1)

# fig.append_trace(go.Scatter(
#     x = np.arange(1,25), 
#     y = dict_of_aggregated_lp.get('VV21').get('ClearedTrade_Info'), 
#     name = 'MCP',
#     line=dict(color=color_book.get('MCP'), width=3),
#     legendgroup = 3
# ), row = num_rows, col = 1)

# for broker in list_of_brokers:
#     for col, i in zip(list_of_collections, range(1,num_rows)):            
            
#         data = dict_of_aggregated_lp.get(broker).get(col)     
#         fig.append_trace(go.Scatter(
#             x = np.arange(1,25), 
#             y = data, 
#             name = broker, 
#             line=dict(color=color_book.get(broker), width=3),
#             legendgroup = i,
#             showlegend=False
#         ), row = i, col = 2)

# # # Update yaxis properties
# for row in range(num_rows):
#     fig.update_yaxes(title_text="Quantity MWh", row=row, col=1)

# fig.update_yaxes(title_text="Price $", row=num_rows, col=1)
# fig.update_yaxes(title_text="Price $", col=2)
# fig.update_xaxes(title_text="Proximity", range=[0, 25])

# fig.update_layout(
#     height=750, 
#     width=1400, 
#     template='ggplot2',
#     title_text="Performance Comparison", 
#     legend_tracegroupgap = 180
# )

# fig.write_html(storage_path + "/overall.html")